<a href="https://colab.research.google.com/github/IMMYIMRAN85/IMMYIMRAN85.github.io/blob/main/toxic_comment_bert_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

 Step 1: Install Hugging Face Transformers & Datasets

In [10]:
!pip install transformers datasets -q



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.4/491.4 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 12.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.3.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cud

 Step 2: Import Required Libraries

In [11]:
import pandas as pd
import torch
from torch.utils.data import DataLoader, TensorDataset
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification
from torch.optim import AdamW
from sklearn.model_selection import train_test_split
from tqdm import tqdm


Mount Google Drive

In [15]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


 Load the CSV from Drive

In [17]:
import pandas as pd

df = pd.read_csv('/content/drive/MyDrive/toxic_data/train.csv')


Combine Labels and Rename for BERT

In [18]:
# Combine all 6 labels into one binary label
label_cols = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']
df['label'] = df[label_cols].max(axis=1)

# Keep only text + label columns
df = df[['comment_text', 'label']].rename(columns={'comment_text': 'text'})

# Drop missing and sample for training
df = df.dropna().sample(5000, random_state=42).reset_index(drop=True)

# Preview
df.head()


,text,label
0,"Geez, are you forgetful! We've already discus...",0
1,Carioca RFA \n\nThanks for your support on my ...,0
2,"""\n\n Birthday \n\nNo worries, It's what I do ...",0
3,Pseudoscience category? \n\nI'm assuming that ...,0
4,"(and if such phrase exists, it would be provid...",0


Tokenize and Split the Dataset

In [19]:
from sklearn.model_selection import train_test_split
from transformers import DistilBertTokenizer
from torch.utils.data import TensorDataset, DataLoader
import torch

# Load tokenizer
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

# Split into train/validation sets
train_texts, val_texts, train_labels, val_labels = train_test_split(
    df['text'], df['label'], test_size=0.2, random_state=42
)

# Tokenize text
train_encodings = tokenizer(list(train_texts), truncation=True, padding=True, max_length=64, return_tensors='pt')
val_encodings = tokenizer(list(val_texts), truncation=True, padding=True, max_length=64, return_tensors='pt')

# Convert labels to tensors
train_labels = torch.tensor(train_labels.values)
val_labels = torch.tensor(val_labels.values)

# Create TensorDatasets
train_dataset = TensorDataset(train_encodings['input_ids'], train_encodings['attention_mask'], train_labels)
val_dataset = TensorDataset(val_encodings['input_ids'], val_encodings['attention_mask'], val_labels)

# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=8)


Final Step: Fine-Tune DistilBERT

In [20]:
from transformers import DistilBertForSequenceClassification
from torch.optim import AdamW
from tqdm import tqdm

# Device check
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"✅ Using device: {device}")

# Load pre-trained DistilBERT model
model = DistilBertForSequenceClassification.from_pretrained(
    'distilbert-base-uncased',
    num_labels=2
).to(device)

# Optimizer
optimizer = AdamW(model.parameters(), lr=5e-5)

# Training loop
model.train()
for epoch in range(2):  # You can increase if time allows
    print(f"\n🔁 Epoch {epoch+1}")
    epoch_loss = 0

    loop = tqdm(train_loader, leave=True)
    for batch in loop:
        input_ids, attention_mask, labels = [b.to(device) for b in batch]

        # Forward pass
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        # Logging
        epoch_loss += loss.item()
        loop.set_description(f"Epoch {epoch+1}")
        loop.set_postfix(loss=loss.item())

    print(f"📉 Avg Loss for Epoch {epoch+1}: {epoch_loss / len(train_loader):.4f}")


✅ Using device: cpu


Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



🔁 Epoch 1


Epoch 1: 100%|██████████| 500/500 [27:10<00:00,  3.26s/it, loss=0.953]


📉 Avg Loss for Epoch 1: 0.1787

🔁 Epoch 2


Epoch 2: 100%|██████████| 500/500 [26:41<00:00,  3.20s/it, loss=0.0487]

📉 Avg Loss for Epoch 2: 0.0939


 After Training: Save Your Model

In [23]:
# Save model and tokenizer to Google Drive
model.save_pretrained('/content/drive/MyDrive/toxic_data/distilbert_finetuned')
tokenizer.save_pretrained('/content/drive/MyDrive/toxic_data/distilbert_finetuned')


('/content/drive/MyDrive/toxic_data/distilbert_finetuned/tokenizer_config.json',
 '/content/drive/MyDrive/toxic_data/distilbert_finetuned/special_tokens_map.json',
 '/content/drive/MyDrive/toxic_data/distilbert_finetuned/vocab.txt',
 '/content/drive/MyDrive/toxic_data/distilbert_finetuned/added_tokens.json')

 Now: Evaluation Code (Accuracy, Precision, Recall, F1)

In [25]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

model.eval()
preds = []
true_labels = []

with torch.no_grad():
    for batch in val_loader:
        input_ids, attention_mask, labels = [b.to(device) for b in batch]
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        predictions = torch.argmax(logits, dim=1)
        preds.extend(predictions.cpu().numpy())
        true_labels.extend(labels.cpu().numpy())

# Calculate metrics
accuracy = accuracy_score(true_labels, preds)
precision, recall, f1, _ = precision_recall_fscore_support(true_labels, preds, average='binary')

print(f"✅ Accuracy: {accuracy:.4f}")
print(f"🎯 Precision: {precision:.4f}")
print(f"🔁 Recall: {recall:.4f}")
print(f"📊 F1 Score: {f1:.4f}")


✅ Accuracy: 0.9580
🎯 Precision: 0.9104
🔁 Recall: 0.6289
📊 F1 Score: 0.7439
